In [7]:
import firebase_admin
from firebase_admin import credentials, db
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# ================= CONFIGURASI =================
# 1. Lokasi & Waktu
LAT = -7.66  # Kebumen
LON = 109.65
# Format: YYYY-MM-DD
TARGET_DATE = "2025-11-29" 
START_TIME_WIB = "03:09:00"
END_TIME_WIB   = "07:46:00"

In [10]:
# 2. Firebase Config
# Ganti dengan path file JSON Anda
CRED_PATH = "/work/staklimjerukagung-firebase-admin.json" 
DB_URL = "https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/"
# Path di database tempat data sensor disimpan
# Misal: sensors/kbm-05/logs
DB_NODE_PATH = "/auto_weather_stat/id-05/data" 

# ================= 1. SETUP KONEKSI =================
# Setup Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate(CRED_PATH)
    firebase_admin.initialize_app(cred, {'databaseURL': DB_URL})

# Setup Open-Meteo
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
client = openmeteo_requests.Client(session=retry_session)

In [13]:
# ================= 2. AMBIL DATA OPEN-METEO =================
def get_open_meteo_hourly():
    url = "https://archive-api.open-meteo.com/v1/archive"
    # Catatan: Karena 2025 adalah masa depan, untuk simulasi ini
    # pastikan endpoint mendukung forecast atau gunakan tanggal masa lalu untuk tes.
    # Jika tanggal > hari ini, gunakan endpoint "forecast" bukan "archive".
    
    params = {
        "latitude": LAT,
        "longitude": LON,
        "start_date": TARGET_DATE,
        "end_date": TARGET_DATE,
        "hourly": ["temperature_2m", "relative_humidity_2m"],
        "timezone": "Asia/Jakarta"
    }
    
    print("🚀 Mengambil data Hourly dari Open-Meteo...")
    try:
        responses = client.weather_api(url, params=params)
        response = responses[0]
    except Exception as e:
        print(f"⚠️ Error Open-Meteo (Mungkin tanggal masa depan?): {e}")
        return None

    # Process Hourly Data
    hourly = response.Hourly()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature": hourly.Variables(0).ValuesAsNumpy(),
        "humidity": hourly.Variables(1).ValuesAsNumpy()
    }
    
    df = pd.DataFrame(data=hourly_data)
    
    # Convert UTC ke WIB (Asia/Jakarta)
    df = df.set_index('date')
    df.index = df.index.tz_convert('Asia/Jakarta')
    return df

# ================= 3. INTERPOLASI KE MENIT =================
def generate_minute_data(df_hourly):
    print("🔄 Melakukan Interpolasi Linear (Hourly -> Minutely)...")
    
    # 1. Resample ke per Menit ('1T')
    # Ini akan membuat baris kosong (NaN) di antara jam 03:00 dan 04:00
    df_minutely = df_hourly.resample('1T').asfreq()
    
    # 2. Interpolasi Linear
    # Mengisi nilai kosong dengan menarik garis lurus antara dua titik jam
    df_minutely = df_minutely.interpolate(method='linear')
    
    # 3. Potong Sesuai Jam Request (03:09 - 07:46)
    start_full = f"{TARGET_DATE} {START_TIME_WIB}"
    end_full   = f"{TARGET_DATE} {END_TIME_WIB}"
    
    # Slicing data
    mask = (df_minutely.index >= start_full) & (df_minutely.index <= end_full)
    df_final = df_minutely.loc[mask]
    
    print(f"✅ Data siap! Total {len(df_final)} titik data (menit).")
    return df_final

# ================= 4. UPDATE KE FIREBASE =================
def update_firebase(df):
    ref = db.reference(DB_NODE_PATH)
    
    print(f"🔥 Mulai upload ke Firebase: {DB_NODE_PATH}")
    count = 0
    
    # Loop setiap baris data
    for index, row in df.iterrows():
        # Format Timestamp sesuai Key di Firebase Anda
        # OPSI A: Jika Key Firebase adalah "YYYY-MM-DD HH:mm:ss"
        key_timestamp = index.strftime('%Y-%m-%d %H:%M:%S')
        
        # OPSI B: Jika Key Firebase adalah Unix Timestamp (angka)
        # key_timestamp = str(int(index.timestamp()))
        
        # Data baru yang mau ditimpa
        new_data = {
            'temperature': round(float(row['temperature']), 2),
            'humidity': round(float(row['humidity']), 2),
            'source': 'replaced_open_meteo' # Opsional: Menandai data hasil edit
        }
        
        # UPDATE data (bukan set/overwrite total, hanya field temp & hum)
        # Pastikan child(key) sesuai dengan struktur DB Anda
        try:
            # Cari node berdasarkan waktu.
            # Jika struktur DB Anda bukan timestamp sebagai key langsung,
            # Anda mungkin perlu query dulu: ref.order_by_child('timestamp').equal_to(key)...
            
            # Asumsi: Key node adalah timestamp string
            ref.child(key_timestamp).update(new_data)
            
            print(f"   Updated: {key_timestamp} -> T:{new_data['temperature']} H:{new_data['humidity']}")
            count += 1
        except Exception as e:
            print(f"❌ Gagal update {key_timestamp}: {e}")

    print(f"🎉 Selesai! {count} data berhasil direplace.")

In [22]:
# ================= MAIN =================
if __name__ == "__main__":
    # 1. Ambil Data Jam-jaman
    df_hourly = get_open_meteo_hourly()
    
    if df_hourly is not None:
        # 2. Ubah jadi Menit & Filter Jam
        df_target = generate_minute_data(df_hourly)
        
        # Cek data sebelum upload
        print(df_target.head())
        print(df_target.tail())
        
        # 3. Eksekusi Upload (Uncomment jika yakin)
        update_firebase(df_target)

🚀 Mengambil data Hourly dari Open-Meteo...
🔄 Melakukan Interpolasi Linear (Hourly -> Minutely)...
✅ Data siap! Total 278 titik data (menit).
                           temperature   humidity
date                                             
2025-11-29 03:09:00+07:00    22.509502  87.934059
2025-11-29 03:10:00+07:00    22.502001  87.933418
2025-11-29 03:11:00+07:00    22.494501  87.932777
2025-11-29 03:12:00+07:00    22.487001  87.932137
2025-11-29 03:13:00+07:00    22.479502  87.931488
                           temperature   humidity
date                                             
2025-11-29 07:42:00+07:00    26.707001  70.081123
2025-11-29 07:43:00+07:00    26.738667  69.935982
2025-11-29 07:44:00+07:00    26.770334  69.790840
2025-11-29 07:45:00+07:00    26.802000  69.645691
2025-11-29 07:46:00+07:00    26.833668  69.500549
🔥 Mulai upload ke Firebase: /auto_weather_stat/id-05/data
   Updated: 2025-11-29 03:09:00 -> T:22.51 H:87.93
   Updated: 2025-11-29 03:10:00 -> T:22.5 H:87.93


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>